In [84]:
import pandas as pd
import yfinance as yf

In [85]:
# df_stock = yf.Ticker('^GSPC').history(start='2004-01-1', interval='1mo').iloc[::12, :]
# df_bond = yf.Ticker('AGG').history(start='2004-01-01', interval='1mo').iloc[::12, :]
# df_comm = yf.Ticker('^SPGSCI').history(start='2004-01-01', interval='1mo').iloc[::12, :]
df_stock = yf.Ticker('AAPL').history(start='1990-01-01', interval='1mo').iloc[::12, :]
df_bond = yf.Ticker('MSFT').history(start='1990-01-01', interval='1mo').iloc[::12, :]
df_comm = yf.Ticker('WMT').history(start='1990-01-01', interval='1mo').iloc[::12, :]
df_portfolio = pd.DataFrame({
    'Stock': df_stock.Close.pct_change(),
    'Bond': df_bond.Close.pct_change(), 
    'Commodity': df_comm.Close.pct_change()}, index=df_stock.index).dropna()
df_portfolio.describe()

,Stock,Bond,Commodity
count,34.000000,34.000000,34.000000
mean,0.347873,0.289717,0.152026
std,0.628591,0.419614,0.290545
min,-0.583132,-0.465781,-0.198816
25%,-0.053124,0.064516,-0.010554
50%,0.262294,0.147034,0.089070
75%,0.667720,0.492471,0.203776
max,2.408689,1.346043,1.171287


In [86]:
RISK_FREE = 0.05

def shrp(w):
    R = np.matmul(df_portfolio.to_numpy(), w)
    return (R.mean()-0.05)/R.std()

In [87]:
import numpy as np

w = np.array([1/3, 1/3, 1/3])
epochs = 10000
learning_rate = 0.001
h = 0.0000001
max_score = 0
max_w = []

for _ in range(epochs):
    s = shrp(w)
    grad = np.zeros(len(w))
    for i, w_i in enumerate(w):
        w_l = w.copy()
        w_r = w.copy()
        w_l[i] = w_i - h
        w_l /= w_l.sum()
        w_r[i] = w_i + h
        w_r /= w_r.sum()
        grad[i] = (shrp(w_r)-shrp(w_l))/(2*h)
    w += learning_rate*grad
    w /= w.sum()
    w[w < 0] = 0
    w /= w.sum()
    if shrp(w) > max_score:
        max_w = w
        max_score = shrp(w)
np.set_printoptions(suppress=True)
print(max_score, max_w)

0.6813289883351005 [0.33105908 0.66894092 0.        ]


In [88]:
import numpy as np
from scipy.optimize import minimize

# Define the function to be optimized
def func(x):
    return -shrp(x)

# Initial guess
x0 = [1/3, 1/3, 1/3]

# Define bounds for x, y, and z (if needed)
bounds = [(0, 1), (0, 1), (0, 1)]

# Minimize the negative of the function to find the maximum
result = minimize(func, x0, bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: sum(x)-1})

# The result object contains the maximum value and the corresponding values of x, y, and z
max_value = -result.fun
max_x, max_y, max_z = result.x

print("Maximum value:", max_value)
print("Values of x, y, z at maximum:", max_x, max_y, max_z)

Maximum value: 0.6813289593355375
Values of x, y, z at maximum: 0.3308720109462439 0.669127989053756 4.119968255444917e-17
